# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project, for example:

import requests
import bs4
import re
import pyarrow.parquet as pq
import pandas as pd
import geopandas as gpd
from skimpy import clean_columns
import sqlalchemy as db
import matplotlib.pyplot as plt
import math
import os
import io

In [2]:
# any general notebook setup, like log formatting

In [3]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
ny_lat0, ny_lon0 = NEW_YORK_BOX_COORDS[0]
ny_lat1, ny_lon1 = NEW_YORK_BOX_COORDS[1]

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Data lookup from taxi_zones.shp

In [4]:
file_geo = gpd.read_file(r"taxi_zones.shp")

In [5]:
# Get the center of each polygon
file_geo = file_geo.to_crs(4326)
file_geo["lon"] = file_geo.centroid.x
file_geo["lat"] = file_geo.centroid.y

# 
file_geo = file_geo[file_geo["lon"] > ny_lon0]
file_geo = file_geo[file_geo["lat"] > ny_lat0]
file_geo = file_geo[file_geo["lon"] < ny_lon1]
file_geo = file_geo[file_geo["lat"] < ny_lat1]

/tmp/ipykernel_1047/1550122043.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  file_geo["lon"] = file_geo.centroid.x
/tmp/ipykernel_1047/1550122043.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  file_geo["lat"] = file_geo.centroid.y


In [6]:
#create dataframe to handle location data
df_location = file_geo[["LocationID"]].copy()
df_location["coordinate"] = pd.Series(zip(file_geo.lat, file_geo.lon))

df_location = df_location.dropna()

df_location

,LocationID,coordinate
0,1,"(40.69183120640149, -74.17400027276298)"
1,2,"(40.61674529165988, -73.83129854302214)"
2,3,"(40.86447368477543, -73.84742223236718)"
3,4,"(40.72375214158478, -73.97696825691767)"
5,6,"(40.76149256216376, -73.91969431946073)"
...,...,...
250,251,"(40.897932025294715, -73.8522154537012)"
251,252,"(40.74423471780149, -73.90630644197886)"
252,253,"(40.70913894067186, -74.01302277174901)"
253,254,"(40.77593240314995, -73.94651035601467)"


### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [7]:
def calculate_distance(from_coord, to_coord):
    
    result = []
    
    for data in range(max(len(from_coord), len(to_coord))):
        lat1, lon1 = from_coord[data]
        lat2, lon2 = to_coord[data]
                      
        #method to convert a degree value into radians
        lon1 = math.radians(lon1)
        lon2 = math.radians(lon2)
        lat1 = math.radians(lat1)
        lat2 = math.radians(lat2)
        
        #Haversine Formula
        diff_lon = lon2 - lon1 
        diff_lat = lat2 - lat1
        
        a = math.sin(diff_lat // 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(diff_lon // 2)**2
        c = 2 * math.asinh(math.sqrt(a))
        
        # Radius of earth in kilometers. Use 3956 for miles
        r = 6371
                      
        #calculate the result
        distance = r*c
        result.append(distance)
    
    distance_series = pd.Series(result)
                      
    return distance_series

In [74]:
def add_distance_column(dataframe):
    #merge main dataframe with the df_location based on Pick-up Location
    dataframe = dataframe.merge(df_location, left_on='PULocationID', right_on='LocationID')
    dataframe = dataframe.rename(columns={'coordinate':'PUCoordinate'})
    
    #merge main dataframe with the df_location based on Drop-off Location
    dataframe = dataframe.merge(df_location, left_on='DOLocationID', right_on='LocationID')
    dataframe = dataframe.rename(columns={'coordinate':'DOCoordinate'})

    dataframe = dataframe.drop(columns=["LocationID_x","LocationID_y"])
    
    dataframe["distance"] = calculate_distance(dataframe["PUCoordinate"], dataframe["DOCoordinate"])
    
    return dataframe["distance"]

### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [54]:
def find_taxi_csv_urls():
    #Get taxi HTML
    response = requests.get(TAXI_URL)
    html = response.content
    
    #Find taxi parquet links
    soup = bs4.BeautifulSoup(html, 'html.parser')
    result = [a['href'] for a in soup.find_all('a', title="Yellow Taxi Trip Records")]
    return result

In [55]:
def get_and_clean_month_taxi_data(url):
    dataframe = pd.DataFrame(None)
    match = re.search(r"\w+2022-01.parquet", url)
    if match is None:
        pass
    else:
        response = requests.get(url)
        
        with open(match.group(), "wb") as f:
            for chunk in response.iter_content(chunk_size=1024): 
                if chunk:
                    f.write(chunk)
                    
        with open(match.group(), "r") as f:
            dataframe = pd.read_parquet(match.group(), engine='pyarrow')
            
    return dataframe

In [56]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    for csv_url in all_csv_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    
    taxi_data = taxi_data.dropna()
    taxi_data["store_and_fwd_flag"].replace(["Y", "N"],
                                             [1,0], inplace= True)
    
    return taxi_data

### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [57]:
def load_and_clean_uber_data(csv_file):
    df_uber = pd.read_csv(csv_file)
    
    #renaming old df without creating a copy 
    df_uber.rename(columns={'key': 'date_time', 'Unnamed: 0': 'location_id'}, inplace=True)
    
    return df_uber

In [58]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    add_distance_column(uber_dataframe)
    return uber_dataframe

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [59]:
def clean_month_weather_data_hourly(csv_file):
    df_weather_hourly = pd.read_csv(csv_file)
    
    return df_weather_hourly

In [60]:
def clean_month_weather_data_daily(csv_file):
    df_weather_daily = pd.read_csv(csv_file)
    
    return df_weather_daily

In [61]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    weather_csv_files = []
    list_dir = os.listdir() #list all file within a directory and store in an array

    for file in list_dir:
        match = re.search(r"(2009|201[012345])_weather.csv", file)
        if match is None:
            pass
        else:
            weather_csv_files.append(match.group())
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [77]:
taxi_data = get_and_clean_taxi_data()
taxi_data["distance"] = add_distance_column(taxi_data)
#uber_data = get_uber_data()
#hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

NameError: name 'UBER_DATA' is not defined

In [76]:
taxi_data.isna().sum()

VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count               0
trip_distance                 0
RatecodeID                    0
store_and_fwd_flag            0
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge          0
airport_fee                   0
distance                 252886
dtype: int64

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)